In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-

import numpy as np
import tensorflow as tf
from numpy import array

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from keras.utils import to_categorical # one-hot encoding

Using TensorFlow backend.


### Text Tokenization

In [3]:
# 주어진 문장을 '단어'로 토큰화 하기
# 케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 불러옴
from tensorflow.keras.preprocessing.text import text_to_word_sequence
 
text = '해보지 않으면 해낼 수 없다.'
 
result = text_to_word_sequence(text)
print("원문: ", text)
print("토큰화:", result)

원문:  해보지 않으면 해낼 수 없다.
토큰화: ['해보지', '않으면', '해낼', '수', '없다']


In [4]:
# 단어 빈도수 세기: 텍스트에서 중요한 역할을 하는 단어를 파악할 수 있다.
# Bag-of-words: 같은 단어끼리 따로 가방에 담은 뒤 각 가방에 몇 개의 단어가 들어있는지 세는 기법

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다.',
       '토큰화 한 결과는 딥러닝에서 사용 할 수 있습니다.',
       ]
 
# 토큰화 함수
token = Tokenizer()             
token.fit_on_texts(docs)       # 토큰화 함수에 문장 적용
  
print("\n단어 카운트:\n", token.word_counts) # 순서는 랜덤
#Tokenizer()의 word_counts 함수는 순서를 기억하는 OrderedDict 클래스를 사용

print("\n문장 카운트: ", token.document_count)
print("\n각 단어가 몇개의 문장에 포함되어 있는지:\n", token.word_docs)
print("\n각 단어에 매겨진 인덱스 값:\n",  token.word_index)


단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 3), ('합니다', 1), ('단어로', 1), ('해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('한', 1), ('결과는', 1), ('사용', 1), ('할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트:  3

각 단어가 몇개의 문장에 포함되어 있는지:
 defaultdict(<class 'int'>, {'텍스트의': 2, '토큰화': 3, '나누어': 1, '단어를': 1, '먼저': 1, '각': 1, '합니다': 1, '단어로': 1, '인식됩니다': 1, '해야': 1, '딥러닝에서': 2, '할': 1, '결과는': 1, '수': 1, '있습니다': 1, '한': 1, '사용': 1})

각 단어에 매겨진 인덱스 값:
 {'토큰화': 1, '텍스트의': 2, '딥러닝에서': 3, '먼저': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '단어로': 9, '해야': 10, '인식됩니다': 11, '한': 12, '결과는': 13, '사용': 14, '할': 15, '수': 16, '있습니다': 17}


### One-hot Encoding

- 단순히 단어의 출현 빈도만으로는 해당 단어가 문장의 어디에서 왔는지, 각 단어의 순서는 어땠는지 등의 정보를 얻을 수 없음. 
- 따라서 단어가 문장의 다른 요소와 어떤 관계를 가지고 있는지 파악하는 방법이 필요함. 이러한 기법 중 원핫인코딩이 가장 기본적인 방법.

In [5]:
text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text])
token.word_index # 우선 Tokenization

{'그': 4, '꿈꾸는': 2, '꿈을': 5, '닮아간다': 6, '오랫동안': 1, '이는': 3}

In [6]:
# 토큰의 인덱스로만 채워진 새로운 배열 만듦
x = token.texts_to_sequences([text])
x

[[1, 2, 3, 4, 5, 6]]

In [7]:
word_size = len(token.word_index) + 1 # 파이썬 배열은 0부터 시작하므로 맨 앞에 0이 추가되어야 하기 때문에 인덱스 수에 하나 추가
x = to_categorical(x, num_classes=word_size)
x

array([[[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]], dtype=float32)

### Word Embedding

- 원-핫 인코딩을 그대로 사용하면 벡터의 길이가 너무 길어진다는 단점이 있다. 따라서 이러한 공간적 낭비를 해결하기 위해 단어임베딩 기법을 사용한다.

- 단어 임베딩은 주어진 배열을 정해진 길이로 압축시킨다. 각 단어 간의 유사도를 계산하기 때문에 단어 임베딩으로 얻은 결과가 밀집된 정보를 가지고 있고 공간의 낭비가 적다. 

- 단어 간 유사도는 **오차역전파**를 이용해 계산한다. 적절한 크기로 배열을 바꾸어 주기 위해 최적의 유사도를 계산하는 학습 과정을 거친다. 이는 케라스의 Embedding()을 이용한다.

In [0]:
model = Sequential()
model.add(Embedding(16,4)) # 입력될 단어는 16개, 임베딩 후 출력되는 벡터 크기는 4

### 실습: 텍스트를 읽고 긍정, 부정 예측하기

In [9]:
# 리뷰 자료 지정
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}

리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [10]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줍니다. (각 데이터의 토큰 개수가 다르기 때문에 학습 데이터의 길이를 동일하게 만듦)
padded_x = pad_sequences(x, 4)  
padded_x

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [ 0, 11, 12, 13],
       [ 0,  0,  0, 14],
       [ 0,  0,  0, 15],
       [ 0,  0, 16, 17],
       [ 0,  0, 18, 19],
       [ 0,  0,  0, 20]], dtype=int32)

In [11]:
#임베딩에 입력될 단어의 수를 지정합니다.
word_size = len(token.word_index) +1
 
# 모델 설정
model = Sequential()

# 임베딩: 총 word_size(word_index에 1 더한 값)개의 단어 집합에서 8개의 결과를 출력함
# input_length: 매번 입력될 단어 수는 4개
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten()) # 1차원으로 변형
model.add(Dense(1, activation='sigmoid')) # 출력값은 1

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6957 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6921 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 998us/step - loss: 0.6902 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 999us/step - loss: 0.6884 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6866 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6848 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6830 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 962us/step - loss: 0.6812 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6794 - accuracy: 0.9000
Epoch 11/20
1/1 [================